In [1]:
!pip install -U gdown

import os

# Créer un dossier pour stocker les données
os.makedirs("/workspace/data", exist_ok=True)

# Télécharger le ZIP dans ce dossier
!gdown --id 18m8Zu0M2zZuaeUIIfoKCZ_N9M7cylxIB -O /workspace/FIFA.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=18m8Zu0M2zZuaeUIIfoKCZ_N9M7cylxIB
From (redirected): https://drive.google.com/uc?id=18m8Zu0M2zZuaeUIIfoKCZ_N9M7cylxIB&confirm=t&uuid=5f36322e-6dd0-4bf9-9a20-a4c6fc2d41d1
To: /content/FIFA.zip
100% 1.41G/1.41G [00:06<00:00, 204MB/s]


In [2]:
import zipfile

zip_path = "/workspace/FIFA.zip"
extract_path = "/workspace/data"

# Extraction
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ FIFA.zip extrait avec succès dans", extract_path)


✅ FIFA.zip extrait avec succès dans /content/data


In [3]:

# ✅ 3. Cloner le repo SAM2 et installer les dépendances
!git clone https://github.com/facebookresearch/sam2.git
%cd sam2
!pip install -e ".[dev]"

# ✅ 4. Télécharger les checkpoints
os.chdir("checkpoints")
!./download_ckpts.sh
os.chdir("..")

fatal: destination path 'sam2' already exists and is not an empty directory.
/content/sam2
Obtaining file:///content/sam2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp311-cp311-linux_x86_64.whl size=13851 sha256=db5935216c709465f23225c6c0c5eb016cc5862c6e8b97b9155df56fac3e6f03
  Stored in directory: /tmp/pip-ephem-wheel-cache-3043iyt4/wheels/76/dc/37/006d341f6080de50c00d031747ee8a1a03f3fb513175bce1c0
Successfully built SAM-2
  Attempting uninstall: SAM-2
    Found existing installation: SAM-2 1.0
    Uninstalling SAM-2-1.0:
      Successfully uninstalled SAM-2-1.0
--2025-05-14 13:54:21--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfi

In [4]:


# ✅ 5. Modifier le fichier YAML de configuration
import yaml
from omegaconf import OmegaConf
from pathlib import Path

os.chdir("sam2")

yaml_path = Path("configs/sam2.1_training/sam2.1_hiera_b+_MOSE_finetune.yaml")
with open(yaml_path, 'r') as f:
    config = OmegaConf.load(f)

img_folder = "/workspace/data/JPEGImages"
gt_folder = "/workspace/data/BinaryMasks"
log_dir = "/workspace/logs"




In [5]:

config.dataset.img_folder = img_folder
config.dataset.gt_folder = gt_folder
config.dataset.file_list_txt = None
config.dataset.multiple = 1

config.scratch.resolution = 1024
config.scratch.num_frames = 4

config.launcher.gpus_per_node = 1
config.trainer.checkpoint.save_freq = 5
config.launcher.experiment_log_dir = log_dir

# ✅ Sauvegarder la nouvelle configuration
with open(yaml_path, 'w') as f:
    f.write(OmegaConf.to_yaml(config))

print("✅ YAML mis à jour avec les chemins RunPod")


✅ YAML mis à jour avec les chemins RunPod


In [16]:
%cd /workspace/sam2/
!touch /workspace/sam2/sam2/configs/sam2.1_training/__init__.py


/content/sam2


In [17]:
# 🔧 Remplacer compose() par un chargement direct dans train.py
!sed -i 's/cfg = compose(config_name=args.config)/cfg = OmegaConf.load(args.config)/' /workspace/sam2/training/train.py


In [19]:
import gc
import torch

gc.collect()              # Forcer la libération CPU
torch.cuda.empty_cache()  # Nettoyer CUDA


In [20]:
%cd /workspace/sam2

# Lancement direct en pointant sur le YAML complet
!python training/train.py \
-c /workspace/sam2/sam2/configs/sam2.1_training/sam2.1_hiera_b+_MOSE_finetune.yaml \
--use-cluster 0 \
--num-gpus 1


/content/sam2
###################### Train App Config ####################
scratch:
  resolution: 1024
  train_batch_size: 1
  num_train_workers: 10
  num_frames: 4
  max_num_objects: 3
  base_lr: 5.0e-06
  vision_lr: 3.0e-06
  phases_per_epoch: 1
  num_epochs: 40
dataset:
  img_folder: /content/data/JPEGImages
  gt_folder: /content/data/BinaryMasks
  file_list_txt: null
  multiplier: 2
  multiple: 1
vos:
  train_transforms:
  - _target_: training.dataset.transforms.ComposeAPI
    transforms:
    - _target_: training.dataset.transforms.RandomHorizontalFlip
      consistent_transform: true
    - _target_: training.dataset.transforms.RandomAffine
      degrees: 25
      shear: 20
      image_interpolation: bilinear
      consistent_transform: true
    - _target_: training.dataset.transforms.RandomResizeAPI
      sizes: ${scratch.resolution}
      square: true
      consistent_transform: true
    - _target_: training.dataset.transforms.ColorJitter
      consistent_transform: true
      br